In [1]:
%reset
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import defmod as dm
import multishapemodule as mdm
import multimodule_usefulfunctions as mm 
import hamiltonian_multishape as ham
import kernels as ker
import model_MultiShapeCirclesTranslation as model

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
r = [3,2]
origin1 = [[-3,0], [6,6]]
origin2 = [[0,3], [6,4]]
nb_points = [10,7]

source = mm.multipleCircles(origin1, r ,nb_points)

In [150]:
## Define Deformation Modules
dim = 2
nb_shapes = 2

# Module 1: Translation of first circle
sigma1 = 10
trans1 = dm.deformationmodules.GlobalTranslation(dim, nb_points[0], sigma1)
trans1 = dm.deformationmodules.Translations(dim, 1, sigma1)


# Module 2: Translation of second circle
sigma2 = 10
trans2 = dm.deformationmodules.Translations(dim, 1, sigma2)
#trans2 = dm.deformationmodules.GlobalTranslation(dim, nb_points[1], sigma2)


# Module 3: Background Module
sigma3 = 1.5
nb_pts = nb_points[0] + nb_points[1]
background = dm.deformationmodules.Translations(dim, nb_pts, sigma3)

# Constraints (identity)
Constr1 = torch.cat([torch.eye(nb_points[0]), torch.zeros([nb_points[0], nb_points[1]]), -torch.eye(nb_points[0]), torch.zeros([nb_points[0], nb_points[1]])], 1)
Constr2 = torch.cat([torch.zeros([nb_points[1], nb_points[0]]), torch.eye(nb_points[1]), torch.zeros([nb_points[1], nb_points[0]]), -torch.eye(nb_points[1]),], 1)
Constr = [Constr1, Constr2]

module_list = [trans1, trans2, background]
multimodule = mdm.MultiShapeModule(module_list, Constr)

In [151]:
gd_list = [source[0], source[1], torch.cat([source[0], source[1]],0)]
controls_list = [torch.zeros(1,2), torch.zeros(1,2), torch.zeros(nb_pts, 2)]

In [152]:
gd19 = tmultimodule.indice_gdorch.cat([mm.computeCenter(source[0]).view(1,2), mm.computeCenter(source[1]).view(1,2), source[0], source[1]],0)
gd = gd19

In [165]:
multimodule.indice_gd


[0, 2, 4, 38]

In [158]:
app_list=multimodule(gd, controls, gd19, multimodule.indice_gd)
multimodule.action(gd, trans2, gd_list[1], controls_list[1])

RuntimeError: size mismatch, m1: [2 x 7], m2: [1 x 2] at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/TH/generic/THTensorMath.cpp:940

In [164]:
gd[multimodule.indice_gd[0]:indice_gd[1]], gd

(tensor([[-3.,  0.],
         [ 6.,  6.]], grad_fn=<SliceBackward>),
 tensor([[-3.0000e+00,  0.0000e+00],
         [ 6.0000e+00,  6.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [-5.7295e-01,  1.7634e+00],
         [-2.0729e+00,  2.8532e+00],
         [-3.9271e+00,  2.8532e+00],
         [-5.4271e+00,  1.7634e+00],
         [-6.0000e+00,  3.6739e-16],
         [-5.4271e+00, -1.7634e+00],
         [-3.9271e+00, -2.8532e+00],
         [-2.0729e+00, -2.8532e+00],
         [-5.7295e-01, -1.7634e+00],
         [ 8.0000e+00,  6.0000e+00],
         [ 7.2470e+00,  7.5637e+00],
         [ 5.5550e+00,  7.9499e+00],
         [ 4.1981e+00,  6.8678e+00],
         [ 4.1981e+00,  5.1322e+00],
         [ 5.5550e+00,  4.0501e+00],
         [ 7.2470e+00,  4.4363e+00]], grad_fn=<CatBackward>))

In [160]:
trans1.action(gd_list[0], trans2, gd_list[1], controls_list[1])

RuntimeError: size mismatch, m1: [10 x 7], m2: [1 x 2] at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/TH/generic/THTensorMath.cpp:940

In [137]:
compound.action(gd19, trans2, source[1], controls_list[1])

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        grad_fn=<CatBackward>),
 [tensor([0., 0., 0., 0.], grad_fn=<ViewBackward>),
  tensor([0., 0., 0., 0.], grad_fn=<ViewBackward>),
  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.], grad_fn=<ViewBackward>)])

In [147]:
gd19.shape, multimodule.indice_gd, 

(torch.Size([19, 2]), [0, 2, 4, 38])

In [64]:
module_list=[trans1, trans2, background]
nb_module = len(module_list)
indice_gd = multimodule.indice_gd
indice_controls = multimodule.indice_controls
points = gd
app_list = []
for i in range(nb_module):
    app_list.append(module_list[i](
        gd[indice_gd[i]:indice_gd[i+1]],
        controls[indice_controls[i]:indice_controls[i+1]],
        points
    ).unsqueeze(0))
    

RuntimeError: size mismatch, m1: [34 x 30], m2: [15 x 2] at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/TH/generic/THTensorMath.cpp:940

In [109]:
indice_gd = multimodule.indice_gd
indice_controls = multimodule.indice_controls
points = gd
i=0
trans2(gd[indice_gd[i]:indice_gd[i+1]],
        controls[indice_controls[i]:indice_controls[i+1]],
        points).unsqueeze(0)


tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]], grad_fn=<UnsqueezeBackward0>)

In [166]:
compound = dm.deformationmodules.Compound([trans1, trans2, background])
gd19 = torch.cat([mm.computeCenter(source[0]).view(1,2), mm.computeCenter(source[1]).view(1,2), source[0], source[1]],0)
comp= compound(gd19 , controls, points)


[tensor([[-3.,  0.],
        [ 6.,  6.]], grad_fn=<SliceBackward>), tensor([[ 0.0000,  0.0000],
        [-0.5729,  1.7634]], grad_fn=<SliceBackward>), tensor([[-2.0729e+00,  2.8532e+00],
        [-3.9271e+00,  2.8532e+00],
        [-5.4271e+00,  1.7634e+00],
        [-6.0000e+00,  3.6739e-16],
        [-5.4271e+00, -1.7634e+00],
        [-3.9271e+00, -2.8532e+00],
        [-2.0729e+00, -2.8532e+00],
        [-5.7295e-01, -1.7634e+00],
        [ 8.0000e+00,  6.0000e+00],
        [ 7.2470e+00,  7.5637e+00],
        [ 5.5550e+00,  7.9499e+00],
        [ 4.1981e+00,  6.8678e+00],
        [ 4.1981e+00,  5.1322e+00],
        [ 5.5550e+00,  4.0501e+00],
        [ 7.2470e+00,  4.4363e+00]], grad_fn=<SliceBackward>)]


In [149]:
compound.indice_gd

[0, 2, 4, 38]